In [31]:
import argparse
import random
import string
import os

class RC5:
    """
    Реализация алгоритма RC5 с поддержкой различных режимов шифрования и дополнений.
    """
    def __init__(self, w, R, key):
        self.w = w  # - длина слова (16, 32 или 64 бита). Это влияет на размер блока данных и ключа.
        self.R = R  # - количество раундов шифрования. Чем больше раундов, тем сильнее шифрование.
        self.key = key  # - ключ шифрования, который используется для генерации массива ключей.
        self.T = 2 * (R + 1)  # Размер массива расширенного ключа
        self.w4 = w // 4  # - четверть длины слова в байтах, используется для вычисления длины блока.
        self.w8 = w // 8  # - длина слова в байтах, то есть длина одной части блока.
        self.mod = 2 ** self.w  # - модуль для арифметических операций (например, 2^w).
        self.mask = self.mod - 1  # - маска для обрезания значений до w бит (например, если w = 32, то mask = 0xFFFFFFFF).
        self.b = len(key)  # Длина ключа в байтах

        # Выравнивание и подготовка ключа
        self.__keyAlign() # - выравнивает ключ шифрования, чтобы он соответствовал длине слова.
        self.__keyExtend() # - расширяет ключ, создавая массив ключей S, который используется для шифрования.
        self.__shuffle() # - перемешивает массив ключей и значения из ключа шифрования L.

    def __lshift(self, val, n):
        """Логический циклический сдвиг влево."""
        n %= self.w # - это выражение гарантирует, что n всегда находится в пределах от 0 до self.w - 1, где self.w — длина слова в битах.
        # (val << n) & self.mask — сдвигает число val на n бит влево, добавляя нули справа. Затем применяется маска 
        # self.mask, чтобы обрезать возможные старшие биты, которые могут выйти за пределы допустимой длины.
        # (val & self.mask) >> (self.w - n) — изначально маскируется число val, оставляя только значащие биты (в пределах длины слова). 
        # Затем выполняется сдвиг вправо на (self.w - n) бит, что как бы переносит биты с левой стороны на правую.
        # Операция | объединяет эти два результата: биты, сдвинутые влево, и биты, перенесенные с правой части на левую.
        return ((val << n) & self.mask) | ((val & self.mask) >> (self.w - n))

    def __rshift(self, val, n):
        """Логический циклический сдвиг вправо для заданного числа val на n бит."""
        n %= self.w
        # (val & self.mask) >> n — маскируется число val, оставляя только нужные биты, и затем выполняется сдвиг вправо на n бит.
        # (val << (self.w - n) & self.mask) — сдвигается число val влево на (self.w - n) бит, тем 
        # самым "перенося" старшие биты в младшие разряды, и применяет маску для ограничения длины.
        # Операция | объединяет оба сдвига: правый сдвиг и сдвиг влево, что позволяет выполнить циклический сдвиг вправо.
        return ((val & self.mask) >> n) | (val << (self.w - n) & self.mask)

    def __const(self):
        """Возвращает константы P и Q в зависимости от длины слова."""
        if self.w == 16:
            return 0xB7E1, 0x9E37
        elif self.w == 32:
            return 0xB7E15163, 0x9E3779B9
        elif self.w == 64:
            return 0xB7E151628AED2A6B, 0x9E3779B97F4A7C15

    def __keyAlign(self):
        """Выравнивание ключа: дополняет ключ до кратной длины слова."""
        # Алгоритм RC5 требует, чтобы ключ состоял из целого числа слов длиной w бит. 
        # Если длина ключа не кратна длине слова, его дополняют нулями.
        # __keyAlign разбивает выровненный ключ на массив L, где каждая часть — слово длиной w бит.
        
        # Если ключ пустой (len(key) == 0), создается массив L длиной 1.
        if self.b == 0:
            self.c = 1
        # Если длина ключа не кратна w8, к нему добавляются нулевые байты (b'\x00').
        elif self.b % self.w8:
            self.key += b'\x00' * (self.w8 - self.b % self.w8)
            self.b = len(self.key)
        # После выравнивания ключ разбивается на слова длиной w8 байт. Эти слова сохраняются в массиве L.
        self.c = self.b // self.w8
        self.L = [0] * self.c
        for i in range(self.b - 1, -1, -1):
            self.L[i // self.w8] = (self.L[i // self.w8] << 8) + self.key[i]

    def __keyExtend(self):
        """Генерация массива расширенного ключа."""
        # Генерирует массив расширенных ключей S размером 2 * (R + 1). 
        # Массив инициализируется двумя магическими константами P и Q, которые зависят от длины слова (w):
        #   P — начальное значение массива, основанное на числе e.
        #   Q — шаг для генерации последующих значений, основанный на числе φ.
        P, Q = self.__const()
        # Массив S заполняется по формуле: S[i] = (P + i * Q) % mod.
        self.S = [(P + i * Q) % self.mod for i in range(self.T)]

    def __shuffle(self):
        """Перемешивание массива ключей S и значений L для генерации итогового ключа."""
        # На этом этапе массив ключей S и массив слов L, полученных из исходного ключа, перемешиваются, 
        # чтобы усилить связь между ключом и итоговым шифром.
        i, j, A, B = 0, 0, 0, 0
        # Перемешивание выполняется 3 * max(len(L), len(S)) шагов.
        for k in range(3 * max(self.c, self.T)):
            # На каждом шаге:
            # Значения из массивов S и L циклически сдвигаются влево (__lshift) на определенное количество бит.
            # Сдвиги зависят от текущих значений в массивах, что делает процесс перемешивания нелинейным.
            A = self.S[i] = self.__lshift((self.S[i] + A + B), 3)
            B = self.L[j] = self.__lshift((self.L[j] + A + B), A + B)
            i = (i + 1) % self.T
            j = (j + 1) % self.c

    def encryptBlock(self, data):
        """Шифрование одного блока данных."""
        # RC5 работает с блоками данных, которые разбиваются на две части: A и B. 
        # Каждый блок проходит через несколько раундов шифрования.
        
        # 1. Инициализация:
        # Данные преобразуются в два числа (A и B) из байтового представления.
        A = int.from_bytes(data[:self.w8], byteorder='little')
        B = int.from_bytes(data[self.w8:], byteorder='little')
        # A и B увеличиваются на первые два ключа из массива S (S[0] и S[1]).
        A = (A + self.S[0]) % self.mod
        B = (B + self.S[1]) % self.mod
        # 2. Раунды шифрования:
        for i in range(1, self.R + 1):
            # В каждом раунде выполняются следующие операции:
            # A обновляется, используя циклический сдвиг влево, XOR с B и сложение с ключом S[2i].
            A = (self.__lshift((A ^ B), B) + self.S[2 * i]) % self.mod
            # B обновляется аналогично, но с использованием A и следующего ключа S[2i+1].
            B = (self.__lshift((A ^ B), A) + self.S[2 * i + 1]) % self.mod
            # Эти операции создают сильную зависимость между двумя частями блока данных.
        # После всех раундов A и B объединяются и возвращаются в виде байтов.
        return (A.to_bytes(self.w8, byteorder='little') + B.to_bytes(self.w8, byteorder='little'))

    def decryptBlock(self, data):
        """Расшифрование одного блока данных."""
        # Процесс расшифрования является обратным шифрованию.
        
        # 1. Инициализация:
        # Данные преобразуются в числа A и B.
        # Операции сложения из шифрования заменяются вычитанием, а сдвиги выполняются вправо.
        A = int.from_bytes(data[:self.w8], byteorder='little')
        B = int.from_bytes(data[self.w8:], byteorder='little')
        # 2. Обратные раунды:
        # Раунды выполняются в обратном порядке — от последнего (R) к первому.
        for i in range(self.R, 0, -1):
            # Для каждого раунда:
            # B восстанавливается через вычитание ключа, циклический сдвиг вправо и XOR с A.
            B = self.__rshift(B - self.S[2 * i + 1], A) ^ A
            # A восстанавливается аналогично, но с использованием B.
            A = self.__rshift(A - self.S[2 * i], B) ^ B
        # После всех раундов из A и B вычитаются первые два ключа (S[0] и S[1]), и результат возвращается как байты.
        B = (B - self.S[1]) % self.mod
        A = (A - self.S[0]) % self.mod
        return (A.to_bytes(self.w8, byteorder='little') + B.to_bytes(self.w8, byteorder='little'))

    def applyPadding(self, data, mode):
        """Применение дополнения к данным в зависимости от указанного режима."""
        padding_length = self.w4 - (len(data) % self.w4)
        if mode == 'ANSI':
            # Добавляем padding_length нулевых байтов (\x00).
            return data + b'\x00' * padding_length
        elif mode == 'X.923':
            # Добавляем (padding_length - 1) нулевых байтов, а в последний байт записывает размер дополнения.
            return data + b'\x00' * (padding_length - 1) + bytes([padding_length])
        elif mode == 'PKCS7':
            # Добавляем padding_length байтов, каждый из которых содержит значение padding_length. 
            return data + bytes([padding_length] * padding_length)
        elif mode == 'ISO7816':
            # Добавляем байт \x80, за которым следуют (padding_length - 1) нулевых байтов.
            return data + b'\x80' + b'\x00' * (padding_length - 1)

    def removePadding(self, data, mode):
        """Удаление дополнения из данных в зависимости от указанного режима."""
        if mode == 'ANSI':
            # Удаляем padding_length нулевых байтов (\x00).
            return data.rstrip(b'\x00')
        elif mode == 'X.923':
            padding_length = data[-1]
            return data[:-padding_length]
        elif mode == 'PKCS7':
            padding_length = data[-1]
            return data[:-padding_length]
        elif mode == 'ISO7816':
            return data.rstrip(b'\x00')[:-1]

    def encryptECB(self, data):
        """Шифрование данных в режиме ECB (Electronic Codebook)."""
        result = b''
        for i in range(0, len(data), self.w4):
            # Данные разбиваются на блоки фиксированной длины (self.w4):
            block = data[i:i+self.w4]
            # Каждому блоку применяется шифрование методом encryptBlock.
            # Блоки шифруются независимо друг от друга.
            result += self.encryptBlock(block)
        return result

    def decryptECB(self, data):
        """Расшифрование данных в режиме ECB (Electronic Codebook)."""
        result = b''
        for i in range(0, len(data), self.w4):
            # Аналогично шифрованию, данные разбиваются на блоки фиксированной длины.
            block = data[i:i+self.w4]
            # Для каждого блока вызывается decryptBlock, который расшифровывает данные.
            result += self.decryptBlock(block)
        return result

    def encryptCBC(self, data, iv):
        """Шифрование данных в режиме CBC (Cipher Block Chaining)."""
        result = b''
        prev_block = iv
        # В начале используется вектор инициализации (IV).
        for i in range(0, len(data), self.w4):
            block = data[i:i+self.w4]
            # Каждый блок данных XOR-ится с результатом предыдущего зашифрованного блока (или IV для первого блока).
            xor_block = bytes(a ^ b for a, b in zip(block, prev_block))
            # После этого результат XOR шифруется с помощью encryptBlock.
            encrypted_block = self.encryptBlock(xor_block)
            result += encrypted_block
            prev_block = encrypted_block
        return result

    def decryptCBC(self, data, iv):
        """Расшифрование данных в режиме CBC (Cipher Block Chaining)."""
        result = b''
        prev_block = iv
        # Начинается с вектора инициализации (IV).
        for i in range(0, len(data), self.w4):
            block = data[i:i+self.w4]
            # Каждый блок сначала расшифровывается с помощью decryptBlock.
            decrypted_block = self.decryptBlock(block)
            # После расшифровки результат XOR-ится с предыдущим зашифрованным блоком (или IV для первого блока).
            result += bytes(a ^ b for a, b in zip(decrypted_block, prev_block))
            prev_block = block
        return result

    def encryptPCBC(self, data, iv): 
        """Шифрование данных в режиме PCBC (Propagating Cipher Block Chaining)."""
        result = b''
        prev_plaintext = iv
        prev_ciphertext = iv
        # Похож на CBC, но есть дополнительное влияние на следующий блок.
        for i in range(0, len(data), self.w4):
            block = data[i:i+self.w4]
            # Каждый исходный блок XOR-ится с предыдущим исходным блоком (plaintext) и предыдущим зашифрованным блоком (ciphertext).
            xor_block = bytes(a ^ b ^ c for a, b, c in zip(block, prev_plaintext, prev_ciphertext))
            # После этого результат XOR шифруется с помощью encryptBlock.
            encrypted_block = self.encryptBlock(xor_block)
            result += encrypted_block
            prev_plaintext = block
            prev_ciphertext = encrypted_block
        return result

    def decryptPCBC(self, data, iv):
        """Расшифрование данных в режиме PCBC (Propagating Cipher Block Chaining)."""
        result = b''
        prev_plaintext = iv
        prev_ciphertext = iv
        # Похож на CBC, но есть дополнительное влияние на следующий блок.
        for i in range(0, len(data), self.w4):
            block = data[i:i+self.w4]
            # Расшифровываем текущий блок
            decrypted_block = self.decryptBlock(block)
            # Каждый расшифрованный блок XOR-ится с предыдущим исходным блоком (plaintext) и предыдущим зашифрованным блоком (ciphertext).
            xor_block = bytes(a ^ b ^ c for a, b, c in zip(decrypted_block, prev_plaintext, prev_ciphertext))
            result += xor_block
            prev_plaintext = xor_block
            prev_ciphertext = block
        return result

    def encryptCFB(self, data, iv):
        """Шифрование данных в режиме CFB (Cipher Feedback Mode)."""
        result = b''
        prev_block = iv
        # Используется вектор инициализации (IV).
        for i in range(0, len(data), self.w4):
            # IV (или предыдущий зашифрованный блок) сначала шифруется с помощью encryptBlock.
            encrypted_block = self.encryptBlock(prev_block)
            block = data[i:i+self.w4]
            # Результат XOR-ится с текущим блоком данных, чтобы получить зашифрованный блок.
            cipher_block = bytes(a ^ b for a, b in zip(block, encrypted_block))
            result += cipher_block
            # Зашифрованный блок становится входом для следующего блока.
            prev_block = cipher_block
        return result

    def decryptCFB(self, data, iv):
        """Расшифрование данных в режиме CFB (Cipher Feedback Mode)."""
        result = b''
        prev_block = iv
        for i in range(0, len(data), self.w4):
            # Вход (IV или предыдущий зашифрованный блок) шифруется с помощью encryptBlock.
            encrypted_block = self.encryptBlock(prev_block)
            block = data[i:i+self.w4]
            # Результат XOR-ится с текущим зашифрованным блоком, чтобы получить исходный блок данных.
            plaintext_block = bytes(a ^ b for a, b in zip(block, encrypted_block))
            result += plaintext_block
            # Текущий зашифрованный блок становится входом для следующего блока.
            prev_block = block
        return result

    def encryptOFB(self, data, iv):
        """Шифрование данных в режиме OFB (Output Feedback Mode)."""
        result = b''
        prev_block = iv
        # Используется вектор инициализации (IV).
        for i in range(0, len(data), self.w4):
            # IV (или предыдущий зашифрованный блок) шифруется с помощью encryptBlock.
            encrypted_block = self.encryptBlock(prev_block)
            prev_block = encrypted_block
            block = data[i:i+self.w4]
            # Результат XOR-ится с текущим блоком данных для получения зашифрованного блока.
            result += bytes(a ^ b for a, b in zip(block, encrypted_block))
            # В отличие от CFB, зашифрованный блок не используется в качестве входа для следующего 
            # шифрования. Вместо этого используется результат шифрования предыдущего блока.
        return result

    def decryptOFB(self, data, iv):
        """Расшифрование данных в режиме OFB (Output Feedback Mode)."""
        # Работает точно так же, как шифрование, потому что XOR обратима.
        return self.encryptOFB(data, iv)
    
    def encryptCTR(self, data, iv):
        """Шифрование данных в режиме CTR (Counter Mode)."""
        result = b''
        counter = self.encryptBlock(iv)
        # Используется вектор инициализации (IV), который играет роль начального значения счетчика.
        for i in range(0, len(data), self.w4):
            block = data[i:i+self.w4]
            # На каждом шаге счетчик шифруется с помощью encryptBlock.
            encrypted_counter = self.encryptBlock(counter)
            # Зашифрованный счетчик XOR-ится с текущим блоком данных для получения зашифрованного блока.
            cipher_block = bytes(a ^ b for a, b in zip(block, encrypted_counter))
            result += cipher_block
            # Счетчик инкрементируется на каждом шаге.
            counter = (int.from_bytes(counter, 'big') + 1) % (2 ** (self.w))
            counter = counter.to_bytes(self.w8, 'big')

        return result

    def decryptCTR(self, data, iv):
        """Расшифрование данных в режиме CTR."""
        # Работает так же, как шифрование, потому что XOR обратим.
        return self.encryptCTR(data, iv)

In [32]:
def generate_in_bin(file_path, size=1024):
    """
    Генерирует файл in.bin с случайным содержимым.
    :param file_path: Путь для сохранения файла.
    :param size: Размер файла в байтах (по умолчанию 1024).
    """
    content = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=size))
    with open(file_path, 'wb') as f:
        f.write(content.encode('utf-8'))
        print("Файл in.bin сгенерирован.")

def generate_key_bin(file_path, size=16):
    """
    Генерирует файл key.bin с случайным ключом.
    :param file_path: Путь для сохранения файла.
    :param size: Размер ключа в байтах (по умолчанию 16).
    """
    key = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=size))
    with open(file_path, 'wb') as f:
        f.write(key.encode('utf-8'))
        print("Файл key.bin сгенерирован.")

def generate_sync_bin(file_path, size=16):
    """
    Генерирует файл sync.bin с случайной синхропосылкой (IV).
    :param file_path: Путь для сохранения файла.
    :param size: Размер синхропосылки в байтах (по умолчанию 16).
    """
    iv = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=size))
    with open(file_path, 'wb') as f:
        f.write(iv.encode('utf-8'))
        print("Файл sync.bin сгенерирован.")

#generate_in_bin("in.bin", size=1024)
#generate_key_bin("key.bin", size=16)
#generate_sync_bin("sync.bin", size=16)


In [33]:
def main():
    modes = ["ECB", "CBC", "PCBC", "CFB", "OFB", "CTR"]
    paddings = ["ANSI", "X.923", "PKCS7", "ISO/IEC 7816-4"]
    operations = ["Зашифровать", "Расшифровать"]

    print("Режимы шифрования:")
    for i, mode in enumerate(modes, 0):
        print(f"{i}. {mode}")

    print("\nРежимы дополнения:")
    for i, padding in enumerate(paddings, 0):
        print(f"{i}. {padding}")

    print("\nДоступные операции:")
    for i, operation in enumerate(operations, 0):
        print(f"{i}. {operation}")

    # Запросить у пользователя пути к файлам
    input_file = input("Введите путь к файлу с входными данными (например, in.bin): ")
    output_file = input("Введите путь для выходного файла (например, out.bin): ")
    #key_file = input("Введите путь к файлу с ключом (например, key.bin): ")
    key_file = "key.bin"
    #iv_file = input("Введите путь к файлу с синхропосылкой (если не требуется, нажмите Enter): ") or None
    iv_file = "sync.bin"

    # Выбрать режим шифрования
    try:
        mode_index = int(input("\nВыберите режим шифрования (0-5): "))
        mode = modes[mode_index]
    except (ValueError, IndexError):
        print("Некорректный выбор режима шифрования.")
        return

    # Выбрать режим дополнения
    try:
        padding_index = int(input("\nВыберите режим дополнения (0-3): "))
        padding = paddings[padding_index]
    except (ValueError, IndexError):
        print("Некорректный выбор режима дополнения.")
        return

    # Выбрать операцию
    try:
        operation_index = int(input("\nВыберите операцию (0-1): "))
        encrypt = operation_index == 0  # True для "Зашифровать", False для "Расшифровать"
    except (ValueError, IndexError):
        print("Некорректный выбор операции.")
        return

    # Выполнить обработку
    try:
        process_file(
            input_file=input_file,
            output_file=output_file,
            key_file=key_file,
            mode=mode,
            iv_file=iv_file,
            encrypt=encrypt,
            padding=padding,
        )
        print(f"\nОперация '{operations[operation_index]}' успешно выполнена. Результат записан в '{output_file}'.")
    except Exception as e:
        print(f"\nОшибка при обработке файла: {e}")


def process_file(input_file, output_file, key_file, mode, iv_file, encrypt, padding):
    """
    Основная функция для обработки файла.
    """
    with open(input_file, 'rb') as infile, open(output_file, 'wb') as outfile, open(key_file, 'rb') as keyfile:
        data = infile.read()
        key = keyfile.read()

        iv = None
        if iv_file:
            with open(iv_file, 'rb') as ivfile:
                iv = ivfile.read()

        rc5 = RC5(w=64, R=12, key=key)

        if encrypt:
            data = rc5.applyPadding(data, mode=padding)

        if mode == "ECB":
            result = rc5.encryptECB(data) if encrypt else rc5.decryptECB(data)
        elif mode == "CBC":
            result = rc5.encryptCBC(data, iv) if encrypt else rc5.decryptCBC(data, iv)
        elif mode == "PCBC":
            result = rc5.encryptPCBC(data, iv) if encrypt else rc5.decryptPCBC(data, iv)
        elif mode == "CFB":
            result = rc5.encryptCFB(data, iv) if encrypt else rc5.decryptCFB(data, iv)
        elif mode == "OFB":
            result = rc5.encryptOFB(data, iv) if encrypt else rc5.decryptOFB(data, iv)
        elif mode == "CTR":
            result = rc5.encryptCTR(data, iv) if encrypt else rc5.decryptCTR(data, iv)
        else:
            raise ValueError("Неизвестный режим шифрования")

        if not encrypt:
            result = rc5.removePadding(result, mode=padding)

        outfile.write(result)

In [34]:
if __name__ == '__main__':
    main()

Режимы шифрования:
0. ECB
1. CBC
2. PCBC
3. CFB
4. OFB
5. CTR

Режимы дополнения:
0. ANSI
1. X.923
2. PKCS7
3. ISO/IEC 7816-4

Доступные операции:
0. Зашифровать
1. Расшифровать

Операция 'Расшифровать' успешно выполнена. Результат записан в 'decrypted.bin'.


In [35]:
def automated_testing():
    modes = ["ECB", "CBC", "PCBC", "CFB", "OFB", "CTR"]
    paddings = ["ANSI", "X.923", "PKCS7", "ISO7816"]
    input_file = "in.bin"
    encrypted_file = "out.bin"
    decrypted_file = "decrypted.bin"
    key_file = "key.bin"
    iv_file = "sync.bin"

    # Чтение исходного файла
    with open(input_file, "rb") as f:
        original_data = f.read()

    # Чтение ключа
    with open(key_file, "rb") as f:
        key = f.read()

    # Чтение синхропосылки (IV)
    with open(iv_file, "rb") as f:
        iv = f.read()

    for mode in modes:
        for padding in paddings:
            print(f"\n=== Режим шифрования: {mode}, режим дополнения: {padding} ===")
            try:
                # Создаем экземпляр RC5
                rc5 = RC5(w=32, R=12, key=key)

                # Применяем дополнение для шифрования, исключение для потоковых режимов
                if mode in ["CFB", "OFB", "CTR"]:
                    padded_data = original_data
                else:
                    padded_data = rc5.applyPadding(original_data, mode=padding)

                # Шифрование
                if mode == "ECB":
                    encrypted_data = rc5.encryptECB(padded_data)
                elif mode == "CBC":
                    encrypted_data = rc5.encryptCBC(padded_data, iv)
                elif mode == "PCBC":
                    encrypted_data = rc5.encryptPCBC(padded_data, iv)
                elif mode == "CFB":
                    encrypted_data = rc5.encryptCFB(original_data, iv)
                elif mode == "OFB":
                    encrypted_data = rc5.encryptOFB(original_data, iv)
                elif mode == "CTR":
                    encrypted_data = rc5.encryptCTR(original_data, iv)
                else:
                    continue

                # Сохраняем зашифрованные данные
                with open(encrypted_file, "wb") as f:
                    f.write(encrypted_data)

                # Расшифрование
                if mode == "ECB":
                    decrypted_padded_data = rc5.decryptECB(encrypted_data)
                elif mode == "CBC":
                    decrypted_padded_data = rc5.decryptCBC(encrypted_data, iv)
                elif mode == "PCBC":
                    decrypted_padded_data = rc5.decryptPCBC(encrypted_data, iv)
                elif mode == "CFB":
                    decrypted_padded_data = rc5.decryptCFB(encrypted_data, iv)
                elif mode == "OFB":
                    decrypted_padded_data = rc5.decryptOFB(encrypted_data, iv)
                elif mode == "CTR":
                    decrypted_padded_data = rc5.decryptCTR(encrypted_data, iv)
                else:
                    continue

                # Удаление дополнения после расшифрования, исключение для потоковых режимов
                if mode in ["CFB", "OFB", "CTR"]:
                    decrypted_data = decrypted_padded_data
                else:
                    decrypted_data = rc5.removePadding(decrypted_padded_data, mode=padding)

                # Сохраняем расшифрованные данные
                with open(decrypted_file, "wb") as f:
                    f.write(decrypted_data)

                # Проверка совпадения
                if original_data == decrypted_data:
                    print("УСПЕХ: Исходные и дешифрованные данные совпадают.")
                else:
                    print("ОШИБКА: Исходные и дешифрованные данные не совпадают.")

            except Exception as e:
                print(f"Ошибка шифрования с режимом {mode} и режимом дополнения {padding}: {e}")

#automated_testing()
